In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import time
import psutil

In [18]:
# Medir o uso de CPU e memória inicial do processo

# Obtém o uso de CPU e memória atual
cpu_usage = psutil.cpu_percent()
memory_usage = psutil.virtual_memory().percent

print(f"Uso de CPU: {cpu_usage}%")
print(f"Uso de memória: {memory_usage}%")

Uso de CPU: 19.3%
Uso de memória: 74.6%


In [19]:
# Medir o tempo de execução do modelo de recomendação baseado em conteúdo
start_time_aplication = time.time()

# Carregar os dados dos filmes
# Tempo para carregar os dados
start_time = time.time()
movies = pd.read_csv('../../data/movies.csv')

# Criar um vetorizador TF-IDF para processar os gêneros dos filmes
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'].fillna(''))

# Calcular a similaridade de cosseno entre os filmes
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Fim da contagem de tempo
end_time = time.time()

# Cálculo do tempo de execução
execution_time = end_time - start_time

print(f"Tempo de execução: {execution_time} segundos")

Tempo de execução: 1.3129942417144775 segundos


In [20]:
import cProfile

def recommend_content_based(movie_title, cosine_sim=cosine_sim, movies=movies):
    # Encontrar o índice do filme fornecido
    idx = movies.index[movies['title'] == movie_title].tolist()[0]

    # Calcular as pontuações de similaridade de cosseno para todos os filmes
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar os filmes com base nas pontuações de similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obter os índices dos 10 filmes mais similares
    movie_indices = [i[0] for i in sim_scores[1:11]]

    # Retornar os títulos dos filmes recomendados
    return movies['title'].iloc[movie_indices]

# Executar o perfil de desempenho do modelo de recomendação baseado em conteúdo
cProfile.run('recommend_content_based("Toy Story (1995)")')

         10869 function calls (10861 primitive calls) in 0.015 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10329    0.002    0.000    0.002    0.000 3910357227.py:11(<lambda>)
        1    0.000    0.000    0.000    0.000 3910357227.py:14(<listcomp>)
        1    0.002    0.002    0.015    0.015 3910357227.py:3(recommend_content_based)
        1    0.001    0.001    0.015    0.015 <string>:1(<module>)
        8    0.000    0.000    0.000    0.000 __init__.py:33(using_copy_on_write)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:132(geterr)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:33(seterr)
        1    0.000    0.000    0.000    0.000 _ufunc_config.py:426(__init__)
        1    0.000    0.000    0.000    0.000 _ufunc_config.py:430(__enter__)
        1    0.000    0.000    0.000    0.000 _ufunc_config.py:435(__exit__)
        1    0.000    0.000    0.000    0.000 algorithms.py:1

In [21]:
# Exemplo: fazer recomendações para o filme "Toy Story (1995)"
movie_title = "Toy Story (1995)"
recommended_movies = recommend_content_based(movie_title)
print(recommended_movies)

# Fim da contagem de tempo da aplicação do modelo de recomendação baseado em conteúdo
end_time_aplication = time.time()

# Cálculo do tempo de execução da aplicação do modelo de recomendação baseado em conteúdo
execution_time_aplication = end_time_aplication - start_time_aplication

print(f"Tempo de execução da aplicação: {execution_time_aplication} segundos")

# Medir o uso de CPU e memória final do processo

# Obtém o uso de CPU e memória atual
cpu_usage = psutil.cpu_percent()
memory_usage = psutil.virtual_memory().percent

print(f"Uso de CPU: {cpu_usage}%")
print(f"Uso de memória: {memory_usage}%")

1815                                          Antz (1998)
2496                                   Toy Story 2 (1999)
2967       Adventures of Rocky and Bullwinkle, The (2000)
3166                     Emperor's New Groove, The (2000)
3811                                Monsters, Inc. (2001)
6617    DuckTales: The Movie - Treasure of the Lost La...
6997                                     Wild, The (2006)
7382                               Shrek the Third (2007)
7987                       Tale of Despereaux, The (2008)
9215    Asterix and the Vikings (Astérix et les Viking...
Name: title, dtype: object
Tempo de execução da aplicação: 1.4700026512145996 segundos
Uso de CPU: 70.1%
Uso de memória: 74.5%
